In [30]:
import os

In [31]:
%pwd

'c:\\Users\\midem\\Blood-Cells-Classification-Using-CNN-DVC-Pipeline'

In [3]:
os.chdir("../")

In [32]:
%pwd

'c:\\Users\\midem\\Blood-Cells-Classification-Using-CNN-DVC-Pipeline'

In [ ]:
# For updating entity 
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [ ]:
# For updating configuration 
from bloodcellClassifier.constants import *
from bloodcellClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [38]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "blood_cells")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE
        )

        return training_config

In [39]:
import time

In [40]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [41]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [43]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [ ]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    #training.model.summary()
    training.train_valid_generator()
    training.train(
       callback_list=callback_list
    )
    
except Exception as e:
    raise e

## Training notebook ends here

In [12]:
config = ConfigurationManager()
prepare_callbacks_config = config.get_prepare_callback_config()
prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

[2024-10-30 21:47:05,225: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-30 21:47:05,225: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-30 21:47:05,237: INFO: common: created directory at: artifacts]
[2024-10-30 21:47:05,239: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-10-30 21:47:05,241: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [13]:
training_config = config.get_training_config()
training = Training(config=training_config)
training.get_base_model()

[2024-10-30 21:47:11,175: INFO: common: created directory at: artifacts\training]
[2024-10-30 21:47:11,445: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


In [14]:
vars(training)

{'config': TrainingConfig(root_dir=WindowsPath('artifacts/training'), trained_model_path=WindowsPath('artifacts/training/model.h5'), updated_base_model_path=WindowsPath('artifacts/prepare_base_model/base_model_updated.h5'), training_data=WindowsPath('artifacts/data_ingestion/blood_cells'), params_epochs=1, params_batch_size=16, params_is_augmentation=True, params_image_size=BoxList([224, 224, 3])),
 'model': <Functional name=functional, built=True>}

In [15]:
train_datag = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=1./255,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.2,
    interpolation_order=1,
    dtype=None
)

In [16]:
#training.config.training_data
#training.config.params_image_size[:-1]
training.config.params_batch_size

16

In [17]:
train_gene = train_datag.flow_from_directory(
    directory=training.config.training_data,
    target_size=[224, 224],
    classes=None,
    batch_size=16,
    shuffle=True,
    seed=123,
    save_to_dir=None,
    follow_links=False,
    subset="training",
    interpolation="bilinear",
    keep_aspect_ratio=False
)

Found 7968 images belonging to 4 classes.


In [18]:
valid_gene = train_datag.flow_from_directory(
    directory=training.config.training_data,
    target_size=[224, 224],
    classes=None,
    batch_size=16,
    shuffle=False,
    seed=123,
    save_to_dir=None,
    follow_links=False,
    subset="validation",
    interpolation="bilinear",
    keep_aspect_ratio=False
)

Found 1989 images belonging to 4 classes.


In [19]:
steps_per_epoch = train_gene.samples // train_gene.batch_size
validation_steps = valid_gene.samples // valid_gene.batch_size
#steps_per_epoch
#validation_steps

In [21]:
training.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

In [22]:
training.model.fit(
            train_gene,
            epochs=1,
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps,
            validation_data=valid_gene,
            callbacks=callback_list
)

498/498 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2661 - loss: 22.1767

c:\Users\midem\.conda\envs\bldvenv2\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


498/498 ━━━━━━━━━━━━━━━━━━━━ 1351s 3s/step - accuracy: 0.2662 - loss: 22.1724 - val_accuracy: 0.2530 - val_loss: 20.6053
